In [21]:
import pandas as pd
import numpy as np
import json
import os

In [22]:
colnames = pd.read_csv('./data_updated.csv', nrows=1).columns
colnames

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'batsman_runs', 'extra_runs',
       'total_runs', 'player_dismissed', 'dismissal_kind', 'id', 'season',
       'city', 'date', 'team1', 'team2', 'toss_winner', 'toss_decision',
       'result', 'dl_applied', 'winner', 'win_by_runs', 'win_by_wickets',
       'player_of_match', 'venue'],
      dtype='object')

In [23]:
df = pd.DataFrame(columns=colnames)

In [24]:
filenames = os.listdir('./raw/ipl_json')[:-1]

In [25]:
def extract_data(json_data, filename, idx):
    columns = [
        "match_id",
        "inning",
        "batting_team",
        "bowling_team",
        "over",
        "ball",
        "batsman",
        "non_striker",
        "bowler",
        "batsman_runs",
        "extra_runs",
        "total_runs",
        "player_dismissed",
        "dismissal_kind",
        "id",
        "season",
        "city",
        "date",
        "team1",
        "team2",
        "toss_winner",
        "toss_decision",
        "result",
        "dl_applied",
        "winner",
        "win_by_runs",
        "win_by_wickets",
        "player_of_match",
        "venue",
    ]
    data = []

    match_id = filename

    result = (
        "normal"
        if "winner" in json_data["info"]["outcome"]
        else json_data["info"]["outcome"]["result"]
    )
    dl_applied = 1 if json_data["info"]["outcome"].get("method") == "D/L" else 0

    win_by_runs = json_data["info"]["outcome"]["by"].get('runs', 0) if "by" in json_data["info"]["outcome"] else 0
    win_by_wickets = json_data["info"]["outcome"]["by"].get('wickets', 0) if "by" in json_data["info"]["outcome"] else 0
    potm = json_data['info'].get("player_of_match", [""])[0]

    winner = (
        json_data["info"]["outcome"].get("winner", "")
        if result == "normal"
        else json_data["info"]["outcome"].get("eliminator", "")
    )

    innings = json_data["innings"]
    for inning_num, inning in enumerate(innings, start=1):
        batting_team = inning["team"]
        bowling_team = [
            team for team in json_data["info"]["teams"] if team != batting_team
        ][0]
        overs = inning["overs"]
        for over in overs:
            over_num = over["over"] + 1
            deliveries = over["deliveries"]
            for ball_num, delivery in enumerate(deliveries, start=1):
                batsman = delivery["batter"]
                non_striker = delivery["non_striker"]
                bowler = delivery["bowler"]
                runs = delivery["runs"]
                batsman_runs = runs["batter"]
                extra_runs = runs["extras"]
                total_runs = runs["total"]

                player_dismissed = None
                dismissal_kind = None
                if "wickets" in delivery:
                    wickets = delivery["wickets"]
                    if wickets:
                        player_dismissed = wickets[0]["player_out"]
                        dismissal_kind = wickets[0]["kind"]

                data.append(
                    [
                        match_id,
                        inning_num,
                        batting_team,
                        bowling_team,
                        over_num,
                        ball_num,
                        batsman,
                        non_striker,
                        bowler,
                        batsman_runs,
                        extra_runs,
                        total_runs,
                        player_dismissed,
                        dismissal_kind,
                        idx,
                        json_data["info"]["season"],
                        json_data["info"].get("city", ""),
                        json_data["info"]["dates"][0],
                        json_data["info"]["teams"][0],
                        json_data["info"]["teams"][1],
                        json_data["info"]["toss"]["winner"],
                        json_data["info"]["toss"]["decision"],
                        result,
                        dl_applied,
                        winner,
                        win_by_runs,
                        win_by_wickets,
                        potm,
                        json_data["info"]["venue"],
                    ]
                )

    return data


In [26]:
for ix, f in enumerate(filenames, start=1):
    with open("./raw/ipl_json/" + f, 'r') as file:
        print(f)
        json_data = json.load(file)
        extracted_data = extract_data(json_data, filename=str.replace(f, ".json", ""), idx=ix)

        df = pd.concat([df, pd.DataFrame(extracted_data, columns=colnames)], axis=0)


1082591.json
1082592.json
1082593.json
1082594.json
1082595.json
1082596.json
1082597.json
1082598.json
1082599.json
1082600.json
1082601.json
1082602.json
1082603.json
1082604.json
1082605.json
1082606.json
1082607.json
1082608.json
1082609.json
1082610.json
1082611.json
1082612.json
1082613.json
1082614.json
1082615.json
1082616.json
1082617.json
1082618.json
1082620.json
1082621.json
1082622.json
1082623.json
1082624.json
1082625.json
1082626.json
1082627.json
1082628.json
1082629.json
1082630.json
1082631.json
1082632.json
1082633.json
1082634.json
1082635.json
1082636.json
1082637.json
1082638.json
1082639.json
1082640.json
1082641.json
1082642.json
1082643.json
1082644.json
1082645.json
1082646.json
1082647.json
1082648.json
1082649.json
1082650.json
1136561.json
1136562.json
1136563.json
1136564.json
1136565.json
1136566.json
1136567.json
1136568.json
1136569.json
1136570.json
1136571.json
1136572.json
1136573.json
1136574.json
1136575.json
1136576.json
1136577.json
1136578.json

## Cleaning

In [27]:
def replace(df, column_search, search_value, column_replace, replace_value):
    df.loc[df[column_search] == search_value, column_replace] = replace_value

def fix(df, column_search, search_value, replace_value):
    df.loc[df[column_search] == search_value, column_search] = replace_value


In [28]:
replace(df, "venue", "Dubai International Cricket Stadium", "city", "Dubai")
replace(df, "venue", "Sharjah Cricket Stadium", "city", "Sharjah")

fix(df, "season", "2007/08", "2008")
fix(df, "season", "2009/10", "2010")
fix(df, "season", "2020/21", "2020")

fix(df, "team1", "Rising Pune Supergiant", "Rising Pune Supergiants")
fix(df, "team2", "Rising Pune Supergiant", "Rising Pune Supergiants")
fix(df, "winner", "Rising Pune Supergiant", "Rising Pune Supergiants")
fix(df, "batting_team", "Rising Pune Supergiant", "Rising Pune Supergiants")
fix(df, "bowling_team", "Rising Pune Supergiant", "Rising Pune Supergiants")
fix(df, "toss_winner", "Rising Pune Supergiant", "Rising Pune Supergiants")

fix(df, "city", "Bangalore", "Bengaluru")

In [29]:
fix(df, "venue", "Arun Jaitley Stadium", "Arun Jaitley Stadium, Delhi")
fix(df, "venue", "Feroz Shah Kotla", "Arun Jaitley Stadium, Delhi")
fix(df, "venue", "Brabourne Stadium", "Brabourne Stadium, Mumbai")
fix(df, "venue", "Dr DY Patil Sports Academy", "Dr DY Patil Sports Academy, Mumbai")
fix(df, "venue", "Eden Gardens", "Eden Gardens, Kolkata")
fix(
    df,
    "venue",
    "Himachal Pradesh Cricket Association Stadium",
    "Himachal Pradesh Cricket Association Stadium, Dharamsala",
)
fix(df, "venue", "M Chinnaswamy Stadium", "M Chinnaswamy Stadium, Bengaluru")
fix(df, "venue", "M.Chinnaswamy Stadium", "M Chinnaswamy Stadium, Bengaluru")
fix(
    df,
    "venue",
    "MA Chidambaram Stadium",
    "MA Chidambaram Stadium, Chepauk, Chennai",
)
fix(
    df,
    "venue",
    "MA Chidambaram Stadium, Chepauk",
    "MA Chidambaram Stadium, Chepauk, Chennai",
)
fix(
    df,
    "venue",
    "Maharashtra Cricket Association Stadium",
    "Maharashtra Cricket Association Stadium, Pune",
)
fix(
    df,
    "venue",
    "Punjab Cricket Association IS Bindra Stadium",
    "Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh",
)
fix(
    df,
    "venue",
    "Punjab Cricket Association IS Bindra Stadium, Mohali",
    "Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh",
)
fix(
    df,
    "venue",
    "Punjab Cricket Association Stadium, Mohali",
    "Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh",
)
fix(
    df,
    "venue",
    "Rajiv Gandhi International Stadium",
    "Rajiv Gandhi International Stadium, Uppal, Hyderabad",
)
fix(
    df,
    "venue",
    "Rajiv Gandhi International Stadium, Uppal",
    "Rajiv Gandhi International Stadium, Uppal, Hyderabad",
)
fix(
    df,
    "venue",
    "Sardar Patel Stadium, Motera",
    "Narendra Modi Stadium, Ahmedabad",
)
fix(df, "venue", "Sawai Mansingh Stadium", "Sawai Mansingh Stadium, Jaipur")
fix(df, "venue", "Wankhede Stadium", "Wankhede Stadium, Mumbai")


In [30]:
df.to_csv("data_updated_2023.csv", index=False)